<a href="https://colab.research.google.com/github/NelsenYoung/GPT_adventure/blob/main/GPT_adventure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Setup of the boilerplate
import torch
import torch.nn.functional as F

!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2026-01-06 21:41:43--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.06s   

2026-01-06 21:41:43 (18.6 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
with open("input.txt") as f:
  text = f.read()
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [3]:
len(text)

1115394

In [4]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [5]:
# Encode the dataset
chars = sorted(list(set(text)))
chars.insert(0, ']')
stoi = {s:i for i, s in enumerate(chars)}
itos = {i:s for i, s in enumerate(chars)}

In [6]:
# Create the character-wise dataset
block_size = 8

X = []
Y = []

# For each token, grab a block
for i in range(len(text) - (block_size)):
  xs = text[i:(i+block_size)]
  ys = text[i+block_size]

  # Encode the block
  xs = [stoi[i] for i in xs]
  ys = [stoi[i] for i in ys]

  X.append(xs)
  Y += ys
  if i % 100000 == 0:
    print(i)


0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000


In [7]:
print(len(X), len(Y))
num_examples = len(X)

1115386 1115386


In [8]:
X = torch.tensor(X)
Y = torch.tensor(Y)

In [9]:
# Split the data into test and train splits
test_train_split = 8
split = (num_examples // 10) * test_train_split
X_train = X[:split]
Y_train = Y[:split]
X_test = X[split:]
Y_test = Y[split:]

In [49]:
# For mini batching, all we want to do is grab 32 random numbers in range 0 and train examlpes size
batch_size = 32
train_data_size = X_train.shape[0]
idxs = torch.randint(train_data_size, (batch_size,))
x = X_train[idxs]
y = Y_train[idxs]
y.shape

torch.Size([32])

In [50]:
num_classes = len(set(text))
emb_dims = 12
num_hidden = 200

model = torch.nn.Sequential(
    torch.nn.Embedding(num_classes, emb_dims),
    torch.nn.Flatten(1),
    torch.nn.Linear(block_size * emb_dims, num_hidden),
    torch.nn.Tanh(),
    torch.nn.Linear(num_hidden, num_classes)
)

logits = model.forward(x)
logits.shape

torch.Size([32, 65])

In [51]:
# Calculate Loss
loss = F.cross_entropy(logits, y)
loss

tensor(4.2140, grad_fn=<NllLossBackward0>)

In [52]:
# Backtrack
loss.backward()
optim = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)
optim.step()